For dimensionality:

low: 2.63 pix / cm

medium: 1.64 pix / cm

high: 4.24 pix / cm

In [1]:
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread, imwrite

import os
import shutil

In [2]:
# dimensions
dims = {
    'low': 2.63,
    'medium': 1.64,
    'top': 4.24
}

In [3]:
def get_channel(path):
    if 'green' in path:
        return 'green'
    elif 'red edge' in path or 'red_edge' in path:
        return 'red edge'
    elif 'nir' in path:
        return 'nir'
    elif 'red' in path:
        return 'red'
    else:
        return 'N/A'

def get_template_dict(level, top, bottom, left, right):
    # delete export folder if it exists and make a new one
    folder_name = f"matching_images/templates"

    # Check if the folder exists before deleting it
    if os.path.exists(folder_name):
        shutil.rmtree(folder_name)
    os.makedirs(folder_name)
    
    # record what level the templates are from
    template_dict = {'level': level}
    
    # path to where the reference images are
    folder_path = f"reflectance_images/v1/{level}"
    # get all channel image paths
    channel_image_paths = [f for f in os.listdir(folder_path) if (os.path.isfile(os.path.join(folder_path, f)) and ('.tif' in f))]
    for image_path in channel_image_paths:
        # open image and transform to np array and extract the reference region
        image = Image.open(f"reflectance_images/v1/{level}/{image_path}")
        image = np.array(image)[top:bottom, left:right]
        # get what channel the image is from
        channel = get_channel(image_path)
        
        # add extracted region to dict
        template_dict[channel] = image
        # export extracted region
        cv2.imwrite(folder_name + f"/{channel}_template.tif", image)
        
    return template_dict

In [4]:
template_dict = get_template_dict("medium", 1400, 5400, 1400, 7300)

"""
template_dict = {
    green: ~_green.tif,
    red: ~_red.tif,
    etc
}
"""

'\ntemplate_dict = {\n    green: ~_green.tif,\n    red: ~_red.tif,\n    etc\n}\n'

In [5]:
def export_matching_level(level, template_dict):
    
    folder_name = f"matching_images/{level}"

    # Check if the folder exists before deleting it
    if os.path.exists(folder_name):
        shutil.rmtree(folder_name)
    # Make all the export folders
    os.makedirs(folder_name)
    folder_name_extract = f"matching_images/{level}/extract"
    folder_name_boxed = f"matching_images/{level}/boxed"
    folder_name_blacked_out = f"matching_images/{level}/blacked_out"
    os.makedirs(folder_name_extract)
    os.makedirs(folder_name_boxed)
    os.makedirs(folder_name_blacked_out)

    # Path to the reference reflectance mosaic images for the level
    image_folder_path = f"reflectance_images/v1/{level}"
    
    # for each image in folder path (ie all the color channels)
    image_paths = [f for f in os.listdir(image_folder_path) if (os.path.isfile(os.path.join(image_folder_path, f)) and ('.tif' in f))]
    for path in image_paths:
        # read image
        image = np.array(Image.open(f'{image_folder_path}/{path}'))
        # check what channel it is 
        channel = get_channel(path)
        # grab the matching template for this channel
        template = template_dict[channel]
        # match it against the template its in
        scale_factor = dims[template_dict['level']] / dims[level]
        # scale the template to the image height
        template = cv2.resize(template, (0, 0), fx=scale_factor, fy=scale_factor)
        # Match the template in the image using normalized cross-correlation
        result = cv2.matchTemplate(image, template, cv2.TM_CCORR_NORMED)
        # Find the location of the best match
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        top_left = max_loc
        bottom_right = (top_left[0] + template.shape[1], top_left[1] + template.shape[0])
        # Extract the matched region from the original image
        matched_region = image[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]] # [top:bottom, left:right]
        
        # Fill the pixels outside the matched region with -1000
        blacked_out = image.copy()
        blacked_out[:top_left[1], :] = -1000  # fill above
        blacked_out[bottom_right[1]:, :] = -1000  # fill below
        blacked_out[:, :top_left[0]] = -1000  # fill to the left
        blacked_out[:, bottom_right[0]:] = -1000  # fill to the right
        
        # Draw a rectangle around the matched region
        cv2.rectangle(image, top_left, bottom_right, 255, 2)

        # Save the matched region separately
        cv2.imwrite(f"{folder_name_extract}/{channel}_template_matched_region.tif", matched_region)
        # save the rectangle bounded image
        cv2.imwrite(f"{folder_name_boxed}/{channel}_template_matched_region_bounded.jpg", image*255)
        # save the blacked out image
        cv2.imwrite(f"{folder_name_blacked_out}/{channel}_template_matched_region_blacked_out.tif", blacked_out)
        
        # print pixel locations of matched region
        print(f"{level} {channel} Template Matched Region:")
        print(f"   Top Left: (x: {top_left[0]}, y: {top_left[1]})")
        print(f"   Bottom Right: (x: {bottom_right[0]}, y: {bottom_right[1]})")
        print()


In [6]:
levels = ['low', 'medium', 'top']

for level in levels:
    export_matching_level(level, template_dict)

/Users/samfields/Library/Python/3.9/lib/python/site-packages/PIL/Image.py:3182: DecompressionBombWarning: Image size (105234801 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


low red edge Template Matched Region:
   Top Left: (x: 3443, y: 4070)
   Bottom Right: (x: 7122, y: 6564)

low red Template Matched Region:
   Top Left: (x: 3443, y: 4070)
   Bottom Right: (x: 7122, y: 6564)

low green Template Matched Region:
   Top Left: (x: 3443, y: 4070)
   Bottom Right: (x: 7122, y: 6564)

low nir Template Matched Region:
   Top Left: (x: 3443, y: 4070)
   Bottom Right: (x: 7122, y: 6564)

medium red edge Template Matched Region:
   Top Left: (x: 1400, y: 1400)
   Bottom Right: (x: 7300, y: 5400)

medium nir Template Matched Region:
   Top Left: (x: 1400, y: 1400)
   Bottom Right: (x: 7300, y: 5400)

medium red Template Matched Region:
   Top Left: (x: 1400, y: 1400)
   Bottom Right: (x: 7300, y: 5400)

medium green Template Matched Region:
   Top Left: (x: 1400, y: 1400)
   Bottom Right: (x: 7300, y: 5400)



/Users/samfields/Library/Python/3.9/lib/python/site-packages/PIL/Image.py:3182: DecompressionBombWarning: Image size (123796077 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


top red edge Template Matched Region:
   Top Left: (x: 5075, y: 4300)
   Bottom Right: (x: 7357, y: 5847)

top red Template Matched Region:
   Top Left: (x: 5074, y: 4301)
   Bottom Right: (x: 7356, y: 5848)

top green Template Matched Region:
   Top Left: (x: 5074, y: 4302)
   Bottom Right: (x: 7356, y: 5849)

top nir Template Matched Region:
   Top Left: (x: 5075, y: 4299)
   Bottom Right: (x: 7357, y: 5846)

